# Notebook zum Vergleichen der Ergebnisse verschiedener Modelle

- Einheitliche Vorverarbeitung
- Random Forest, Entscheidungsbaum und GBM, um nicht nur die XAI-Ansätze sondern auch deren Ergebnisse für verschiedene Modelle zu betrachten
- Untersucht Laufzeiten, globale Feature Importances zwischen den Algorithmen sowie 
Unterschiede zwischen den verglichenen Modellen

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import lime
import lime.lime_tabular
import time

LMFB


## Laden der Daten und Vorverarbeitung

In [2]:
# Daten laden Label formatieren
df = pd.read_csv('data/BankChurners.csv')#.sample(300)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop(df.columns[-2:], axis=1, inplace=True)

df.loc[df['Attrition_Flag'] == "Existing Customer",["Attrition_Flag"]] = 0
df.loc[df['Attrition_Flag'] == "Attrited Customer",["Attrition_Flag"]] = 1

df[["Attrition_Flag"]] = df[["Attrition_Flag"]].astype(int)

In [3]:
# Numpy-Seed festlegen für Reproduzierbarkeit
np.random.seed(0)

#kategorische Variablen werden kodiert
categorical = df.select_dtypes(exclude=["number","bool_"]).columns

encoded = pd.get_dummies(df[categorical], prefix=categorical)
df_enc = pd.concat([encoded, df], axis=1)
df_enc.drop(categorical, axis=1, inplace=True)

#Trennen von Features und Labels
X = df_enc.drop(["Attrition_Flag"], axis=1)
y = df_enc["Attrition_Flag"]

#Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Training der zu evaluierenden Modelle
- alle Modelle liefern gute Ergebnisse

In [4]:
def train_and_store_model(model, X_train, y_train, X_test):
    model.fit(X_train.values, y_train)
    y_pred = model.predict(X_test)
    return {'model': model, 'predictions': y_pred}

models = {
    'RandomForest': RandomForestClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
}

# leeres Dictionary für die Modelle
models_dict = {}

for model_name, model in models.items():
    models_dict[model_name] = train_and_store_model(model, X_train, y_train, X_test)

for model_name, model_info in models_dict.items():
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(y_test, model_info['predictions']))

X has feature names, but RandomForestClassifier was fitted without feature names
X has feature names, but DecisionTreeClassifier was fitted without feature names


Classification Report for RandomForest:

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1717
           1       0.92      0.74      0.82       309

    accuracy                           0.95      2026
   macro avg       0.94      0.87      0.90      2026
weighted avg       0.95      0.95      0.95      2026

Classification Report for DecisionTree:

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1717
           1       0.78      0.77      0.78       309

    accuracy                           0.93      2026
   macro avg       0.87      0.87      0.87      2026
weighted avg       0.93      0.93      0.93      2026

Classification Report for GradientBoosting:

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1717
           1       0.94      0.85      0.89       309

    accuracy                           0.97      2026
 

X has feature names, but GradientBoostingClassifier was fitted without feature names


## SHAP
- iteriert lokal über alle Kombinationen von Features
- lokale Vorhersagen werden global gemittelt 

In [ ]:
shap_values_dict = {}

# Timer starten
start_time = time.time()

# SHAP-Werte für jedes Modell berechnen
for model_name, model_info in models_dict.items():
    model = model_info['model']
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    # Reshapen
    if isinstance(shap_values, list):
        shap_values = shap_values[0]

    shap_values_dict[model_name] = shap_values

# Zeit stoppen
shap_time = time.time() - start_time

# Dataframe für die aggregierten SHAP-Werte
shap_mean_df = pd.DataFrame()

# Aggregierte SHAP-Werte für jedes Modell berechnen
for model_name, shap_values in shap_values_dict.items():
    shap_mean_df[model_name] = np.abs(shap_values).mean(axis=0)

    #Normalisieren der Werte, um sie vergleichbar zu machen
    shap_mean_df[model_name] = shap_mean_df[model_name] / shap_mean_df[model_name].sum()

shap_mean_df.index = X_test.columns
shap_mean_df = shap_mean_df.sort_values(by=list(shap_mean_df.columns), ascending=False)
shap_mean_df.head(10)

## LIME
- trainiert lokale Modelle, um Vorhersagen zu erklären
- lokale Vorhersagen werden global gemittelt 

In [6]:
# Timer starten
start_time = time.time()

explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns)

# LIME-Ergebnisse für jedes Modell berechnen
lime_results_dict = {}

for model_name, model_info in models_dict.items():
    model = model_info['model']
    lime_results = []

    # Über alle Testdaten iterieren
    for instance in X_test.values:
        explanation = explainer.explain_instance(instance, model.predict_proba, num_features=len(X_test.columns))
        lime_results.append(explanation.as_list())

    lime_results_dict[model_name] = lime_results

# Zeit stoppen
lime_time = time.time() - start_time

# Durchschnittliche LIME-Ergebnisse für jedes Modell berechnen
average_lime_results = {}

for model_name, lime_results in lime_results_dict.items():
    average_weights = {}
    for lime_result in lime_results:
        for feature, weight in lime_result:
            average_weights.setdefault(feature, []).append(weight)

    average_lime_results[model_name] = [(feature, np.mean(weights)) for feature, weights in average_weights.items()]

In [ ]:
# Dataframe für die aggregierten LIME-Ergebnisse
lime_mean_df = pd.DataFrame()

for model_name, lime_results in average_lime_results.items():
    lime_mean_df[model_name] = [weight for feature, weight in lime_results]
    
    # Normalisieren der Werte, um sie vergleichbar zu machen
    lime_mean_df[model_name] = lime_mean_df[model_name] / lime_mean_df[model_name].sum()

lime_mean_df.index = [feature for feature, weight in lime_results]

lime_mean_df = lime_mean_df.sort_values(by=list(lime_mean_df.columns), ascending=False)
lime_mean_df

In [ ]:
# Mapping von Indizes zu Features, um die Namen der Features in lime_mean_df anzupassen
index_to_feature = {}
for index in lime_mean_df.index:
    for feature_name in X_test.columns:
        if feature_name in str(index):
            index_to_feature[index] = feature_name

lime_mean_df.index = lime_mean_df.index.map(lambda x: index_to_feature.get(x, x))

# Features werden aggregiert und normalisiert
lime_mean_df = lime_mean_df.abs()
lime_mean_df = lime_mean_df.groupby(lime_mean_df.index).sum()
lime_mean_df = lime_mean_df / lime_mean_df.sum()

lime_mean_df.head(10)

# DICE
- manipuliert die Werte einzelner Features ("Counterfactuals") und beobachtet Veränderungen
- lokale Vorhersagen werden global gemittelt

In [9]:
import dice_ml

# Timer starten
start_time = time.time()

# One-Hot-Encoding für kategorische Variablen
categorical = [feature for feature in X.columns if type(feature) not in ['bool', 'object', 'str']]
continous_features = [feature for feature in X.filter(categorical) if X[feature].max() > 2]

# Dataframe wird in ein Dice-Data-Objekt umgewandelt
data_dice = dice_ml.Data(dataframe=df_enc, continuous_features=continous_features, outcome_name='Attrition_Flag')

# Dictionary für die Dice-Modelle
dice_models = {}

for model_name, model_info in models_dict.items():
    model = model_info['model']
    dice_model = dice_ml.Model(model=model, backend='sklearn')
    dice_models[model_name] = dice_model

In [ ]:
# Sklearn-Warnungen unterdrücken
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn

# Dataframe für die Dice-Ergebnisse
dice_results_dict = {}

for model_name, dice_model in dice_models.items():
    explainer = dice_ml.Dice(data_dice, dice_model, method='random')
    #cfs = explainer.generate_counterfactuals(X_test, total_CFs=3)
    cfs = explainer.global_feature_importance(X_test, total_CFs=10, posthoc_sparsity_param=None)
    dice_results_dict[model_name] = cfs.summary_importance

# Zeit stoppen
dice_time = time.time() - start_time

In [ ]:
dice_mean_df = pd.DataFrame()

for model_name, dice_results in dice_results_dict.items():
    dice_mean_df[model_name] = dice_results
    
    # Normalisieren der Werte, um sie vergleichbar zu machen
    dice_mean_df[model_name] = dice_mean_df[model_name] / dice_mean_df[model_name].sum()

dice_mean_df.index = X_test.columns
dice_mean_df = dice_mean_df.sort_values(by=list(dice_mean_df.columns), ascending=False)
dice_mean_df.head(10)

## Vergleich der Laufzeiten
- SHAP führt mit Abstand, 10x so schnell wie LIME
- DICE ca. 2x langsamer als LIME

In [17]:
time_df = pd.DataFrame({
    'time': [shap_time, lime_time, dice_time],
    'method': ['SHAP', 'LIME', 'Dice'],
})

time_fig = px.bar(time_df, x='method', y='time', title='Vergleich der Laufzeiten')
time_fig.show()

# Abspeichern der Grafik
time_fig.write_html("Images/runtime_comparison.html")

## Visualisierung der Modellergebnisse
- Vorhersagen sind zwischen den verglichenen Modellen relativ einheitlich
- SHAP und LIME sagen relativ ähnliche Feature Importances für die größten Einflüsse heraus
- Features mit weniger Einfluss werden von SHAP und LIME teilweise anders gerankt
- DICE sagt komplett andere Features als einflussreich vorher, hier vor allem Geschlecht und Abschluss
- Vermutung: Generierte Counterfactuals sind besonders besonders unplausibel/unrealistisch und verwirren das Modell

In [ ]:
# Füge die Dataframes zusammen
result_df = pd.concat([shap_mean_df.assign(Method='SHAP'), lime_mean_df.assign(Method='LIME')], axis=0)
result_df = pd.concat([result_df, dice_mean_df.assign(Method='DICE')], axis=0).reset_index().rename(columns={'index': 'Feature'})

# Schmelze die Dataframes für die Visualisierung
result_df = result_df.melt(id_vars=['Feature', 'Method'], var_name='Model', value_name='Feature Importance')
result_df

In [19]:
# Säulendiagramm mit 3 Facetten erstellen
fig = px.bar(result_df.sort_values(by='Feature Importance', ascending=True),
                y='Feature', 
                x='Feature Importance', 
                color='Method', 
                barmode='group',
                facet_col='Model', 
                title="Feature Importance je Modell und XML-Methode",
                height=1000,)
fig.show()

# Abspeichern der Grafik
fig.write_html("Images/model_comparison.html")